# Grammy Lyrics Analysis

### Importing Library and review of the dataset

In [ ]:
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
df = pd.read_csv("/content/sample_data/lyrics.csv", encoding='latin1')

In [ ]:
df.head()

,year,title,published_at,updated_at,category,nominee,artist,url,lyrics,workers,img,winner
0,1959,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Song Of The Year,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,https://www.google.com/search?q=Nel+Blu+Dipint...,I think that a similar dream will not come aga...,"Domenico Modugno, songwriter (Domenico Modugno)",https://www.grammy.com/sites/com/files/styles/...,True
1,1960,2nd Annual GRAMMY Awards (1959),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Song Of The Year,The Battle Of New Orleans,Jimmy Driftwood,https://www.google.com/search?q=The+Battle+Of+...,"Well, in 18 and 14 we took a little trip\nAlon...","Jimmy Driftwood, songwriter (Jimmy Driftwood)",NaN,True
2,1961,3rd Annual GRAMMY Awards (1960),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,Song Of The Year,Theme From Exodus,Ernest Gold,https://www.google.com/search?q=Theme+From+Exo...,"This land is mine, God gave this land to me\nT...","Ernest Gold, songwriter (Ernest Gold)",NaN,True
3,1962,4th Annual GRAMMY Awards (1961),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,Song Of The Year,Moon River,Henry Mancini,https://www.google.com/search?q=Moon+River+Hen...,"Moon river, wider than a mile\nI'm crossing yo...","Henry Mancini & Johnny Mercer, songwriters (He...",https://www.grammy.com/sites/com/files/styles/...,True
4,1963,5th Annual GRAMMY Awards (1962),2017-11-28T00:03:45-08:00,2019-09-10T01:09:02-07:00,Song Of The Year,What Kind Of Fool Am I,"Sammy Davis, Jr. AND ALSO Anthony Newley",https://www.google.com/search?q=What+Kind+Of+F...,What kind of fool am I\nWho never fell in love...,"Leslie Bricusse & Anthony Newley, songwriters ...",https://www.grammy.com/sites/com/files/styles/...,True


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          67 non-null     int64 
 1   title         67 non-null     object
 2   published_at  63 non-null     object
 3   updated_at    63 non-null     object
 4   category      67 non-null     object
 5   nominee       67 non-null     object
 6   artist        67 non-null     object
 7   url           63 non-null     object
 8   lyrics        67 non-null     object
 9   workers       67 non-null     object
 10  img           46 non-null     object
 11  winner        67 non-null     bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 5.9+ KB


### Text Analysis

In [ ]:
# Clean lyrics data

import string
def text_cleansing(df):
    '''Removes brackets, replaces new line breaks with spaces,
    lowercases everything, removes punctuations, extra whitespaces, and break words'''
    df = df.str.replace("[\(\[].*?[\)\]]", '')
    df = df.str.replace("\n", ' ')
    df = df.str.lower()
    df = df.str.replace('[{}]'.format(string.punctuation), '')
    df = df.str.replace(' +', ' ')
    return df

df.loc[:,'clean_ver'] = text_cleansing(df.loc[:,'lyrics'])

<ipython-input-7-9394bc8f4285>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.str.replace("[\(\[].*?[\)\]]", '')
<ipython-input-7-9394bc8f4285>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.str.replace('[{}]'.format(string.punctuation), '')
<ipython-input-7-9394bc8f4285>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.str.replace(' +', ' ')


In [ ]:
print(df["clean_ver"])

0     i think that a similar dream will not come aga...
1     well in 18 and 14 we took a little trip along ...
2     this land is mine god gave this land to me thi...
3     moon river wider than a mile im crossing you i...
4     what kind of fool am i who never fell in love ...
                            ...                        
62    white shirt now red my bloody nose sleepin you...
63    nah starting a war screaming peace at the same...
64    said baby said baby said baby what you doing w...
65    i watched him circle round the block finally s...
66    i used to float now i just fall down i used to...
Name: clean_ver, Length: 67, dtype: object


In [ ]:
# Define the Stopwords

stop_words = ["ayy", "oh", "da", "dee", "ladada", "dada", "dadada", "whoa", "mmm", "yeah", "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", "it", "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"]

In [ ]:
ignore = set(stop_words)

def remove_stopwords(text):
    clean_text = []

    for i in text:
        if isinstance(i, str):  # Check if the value is a string
            words = nltk.word_tokenize(i)
            words = [w for w in words if w not in ignore and len(w) > 1]
            lyric = " ".join(words)
            clean_text.append(lyric)
        else:
            clean_text.append('')  # Replace 'float' values with an empty string

    return clean_text

df['LyricsClean'] = remove_stopwords(df['clean_ver'])

In [ ]:
print(df["LyricsClean"])

0     think similar dream come again pienso que un s...
1     well 18 14 we took little trip along colonel j...
2     land mine god gave land me brave ancient land ...
3     moon river wider than mile im crossing you sty...
4     what kind fool am who never fell love seems im...
                            ...                        
62    white shirt now red my bloody nose sleepin you...
63    nah starting war screaming peace same time all...
64    said baby said baby said baby what you doing w...
65    watched him circle round block finally stopped...
66    used float now just fall down used know im sur...
Name: LyricsClean, Length: 67, dtype: object


In [ ]:
sentimentid = SentimentIntensityAnalyzer()
comp_score = []
sentiment = []

for i in df.loc[:,'LyricsClean']:
    sentiment.append(sentimentid.polarity_scores(i))

df.loc[:,'sent_scores'] = sentiment
df.loc[:,'comp_score'] = df.loc[:,'sent_scores'].apply(lambda x: x['compound'])
df.loc[:,'sentiment'] = df.loc[:,'comp_score'].apply(lambda x: 'Positive' if x>=0.5 else 'Negative' if x<=-0.5 else 'Neutral')

df.head() # Preview the dataset

,year,title,published_at,updated_at,category,nominee,artist,url,lyrics,workers,img,winner,clean_ver,LyricsClean,sent_scores,comp_score,sentiment
0,1959,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Song Of The Year,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,https://www.google.com/search?q=Nel+Blu+Dipint...,I think that a similar dream will not come aga...,"Domenico Modugno, songwriter (Domenico Modugno)",https://www.grammy.com/sites/com/files/styles/...,True,i think that a similar dream will not come aga...,think similar dream come again pienso que un s...,"{'neg': 0.033, 'neu': 0.858, 'pos': 0.11, 'com...",0.9862,Positive
1,1960,2nd Annual GRAMMY Awards (1959),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Song Of The Year,The Battle Of New Orleans,Jimmy Driftwood,https://www.google.com/search?q=The+Battle+Of+...,"Well, in 18 and 14 we took a little trip\nAlon...","Jimmy Driftwood, songwriter (Jimmy Driftwood)",NaN,True,well in 18 and 14 we took a little trip along ...,well 18 14 we took little trip along colonel j...,"{'neg': 0.118, 'neu': 0.776, 'pos': 0.105, 'co...",-0.9573,Negative
2,1961,3rd Annual GRAMMY Awards (1960),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,Song Of The Year,Theme From Exodus,Ernest Gold,https://www.google.com/search?q=Theme+From+Exo...,"This land is mine, God gave this land to me\nT...","Ernest Gold, songwriter (Ernest Gold)",NaN,True,this land is mine god gave this land to me thi...,land mine god gave land me brave ancient land ...,"{'neg': 0.112, 'neu': 0.584, 'pos': 0.304, 'co...",0.9605,Positive
3,1962,4th Annual GRAMMY Awards (1961),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,Song Of The Year,Moon River,Henry Mancini,https://www.google.com/search?q=Moon+River+Hen...,"Moon river, wider than a mile\nI'm crossing yo...","Henry Mancini & Johnny Mercer, songwriters (He...",https://www.grammy.com/sites/com/files/styles/...,True,moon river wider than a mile im crossing you i...,moon river wider than mile im crossing you sty...,"{'neg': 0.0, 'neu': 0.894, 'pos': 0.106, 'comp...",0.6369,Positive
4,1963,5th Annual GRAMMY Awards (1962),2017-11-28T00:03:45-08:00,2019-09-10T01:09:02-07:00,Song Of The Year,What Kind Of Fool Am I,"Sammy Davis, Jr. AND ALSO Anthony Newley",https://www.google.com/search?q=What+Kind+Of+F...,What kind of fool am I\nWho never fell in love...,"Leslie Bricusse & Anthony Newley, songwriters ...",https://www.grammy.com/sites/com/files/styles/...,True,what kind of fool am i who never fell in love ...,what kind fool am who never fell love seems im...,"{'neg': 0.297, 'neu': 0.45, 'pos': 0.253, 'com...",-0.6012,Negative


In [ ]:
# Saving the results to Analysis file

df.to_csv('analysis.csv', index=False)